In [41]:
import os
import sys
import json
import tqdm
from dotenv import load_dotenv
load_dotenv()

project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

import core.data_schema.model as model
from core.database import Database
db = Database()

In [40]:
import pprint as pp
gallery = model.Gallery.objects().first()
# output_data = pop_gallery.to_json()
print(gallery.gid)
for m in gallery.models:
  print(f'name: {m.name}')
  print('tags: ')
  for t in m.tags:
    print(t.name)



72143467
name: Kristof Cale
tags: 
name: Eveline Dellai
tags: 


In [51]:
model_name='Ashley Adams'
model_id = "647c36ffa6f57b54a3e18159"
galleries = model.Gallery.objects(models=model_id)
for g in galleries:
  print(g.id)
  print(g.path)  
  print(g.gid)
  print("------------------")
  
# model.Model.objects(id=model_id)

64829b1b2e33334d7f99c71c
pronpics/10004207
10004207
------------------
64829b1c2e33334d7f99c84c
pronpics/10017411
10017411
------------------
64829b232e33334d7f99cded
pronpics/10131497
10131497
------------------
64829b272e33334d7f99cf6d
pronpics/10173883
10173883
------------------
64829b382e33334d7f99d5cb
pronpics/10400248
10400248
------------------
64829b402e33334d7f99d885
pronpics/10507791
10507791
------------------
64829b442e33334d7f99d9db
pronpics/10567152
10567152
------------------
64829b482e33334d7f99db35
pronpics/10626585
10626585
------------------
64829b622e33334d7f99e362
pronpics/10977144
10977144
------------------
64829b762e33334d7f99e962
pronpics/11253588
11253588
------------------
64829b782e33334d7f99e9ca
pronpics/11272585
11272585
------------------
64829b7c2e33334d7f99eb1b
pronpics/11336591
11336591
------------------
64829b8e2e33334d7f99f03b
pronpics/11595014
11595014
------------------
64829bad2e33334d7f99f8d6
pronpics/12006106
12006106
------------------
64829b